In [1]:
# !pip install geopandas shapely fiona pyogrio

In [4]:
# import os
# import math
# import glob
# import geopandas as gpd
# import pandas as pd
# from shapely.geometry import box
# import fiona
# import warnings

# # Suppress warnings
# warnings.filterwarnings('ignore')

# # ================= CONFIGURATION =================
# # 1. Folder containing your YOLO shapefiles
# #INPUT_YOLO_FOLDER = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\outputRESULTS"
# INPUT_YOLO_FOLDER = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\outputRESULTS"

# # 2. Path to the VARI layer
# #VARI_GDB_PATH = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot24\vari2024.gdb"
# VARI_GDB_PATH = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\plot2022\VARI2022.gdb"
# #VARI_LAYER_NAME = "VariIndex2024v3"
# VARI_LAYER_NAME = "VARI2022"

# # 3. Path to the Plot Boundaries
# #INPUT_PLOT_GDB = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot24\CorrectedPlotDimensions.gdb"
# INPUT_PLOT_GDB = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\plot2022\CorrectedPlotDimensions.gdb"

# # 4. Output Folder
# OUTPUT_FOLDER = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\outputRESULTS\correctedDetections\2022"

# # 5. Area settings
# SUBDIVIDE_THRESHOLD = 0.028 # Subdivide anything larger than this
# MIN_AREA_THRESHOLD = 0.0018  # Eliminate anything smaller than this
# ELIMINATION_PASSES = 3       # How many times to try eliminating small polygons      
# # =================================================

# def subdivide_polygon(geometry, threshold_area):
#     """
#     Splits a large polygon into smaller chunks using a grid (fishnet) approach.
#     """
#     if geometry.area <= threshold_area:
#         return [geometry]
    
#     cell_size = math.sqrt(threshold_area)
#     minx, miny, maxx, maxy = geometry.bounds
#     chunks = []
    
#     x = minx
#     while x < maxx:
#         y = miny
#         while y < maxy:
#             grid_cell = box(x, y, x + cell_size, y + cell_size)
#             chunk = geometry.intersection(grid_cell)
            
#             if not chunk.is_empty:
#                 if chunk.geom_type == 'MultiPolygon':
#                     chunks.extend(list(chunk.geoms))
#                 elif chunk.geom_type in ['Polygon', 'GeometryCollection']:
#                     chunks.append(chunk)
#             y += cell_size
#         x += cell_size
        
#     return chunks

# def eliminate_pass(gdf, min_area, pass_num):
#     """
#     Single pass of elimination: Explode -> Identify Small -> Merge to Nearest Large.
#     """
#     # Explode
#     gdf_exploded = gdf.explode(index_parts=False).reset_index(drop=True)
    
#     gdf_exploded['temp_area'] = gdf_exploded.geometry.area
#     small_polys = gdf_exploded[gdf_exploded['temp_area'] < min_area].copy()
#     large_polys = gdf_exploded[gdf_exploded['temp_area'] >= min_area].copy()
    
#     if small_polys.empty:
#         return gdf_exploded.drop(columns=['temp_area'])
    
#     if large_polys.empty:
#         return gdf_exploded.drop(columns=['temp_area'])

#     print(f"    - Pass {pass_num}: Eliminating {len(small_polys)} tiny polygons...")

#     large_polys['merge_id'] = range(len(large_polys))
    
#     joined = gpd.sjoin_nearest(small_polys, large_polys[['geometry', 'merge_id']], how='left', distance_col='dist')
#     small_polys_mapped = joined[['geometry', 'merge_id']].dropna()
    
#     combined = pd.concat([large_polys[['geometry', 'merge_id']], small_polys_mapped])
#     dissolved = combined.dissolve(by='merge_id')
    
#     return dissolved.reset_index(drop=True)

# def main():
#     print("=== Starting Plant Refinement (Refine + Join Attributes) ===")
    
#     if not os.path.exists(OUTPUT_FOLDER):
#         os.makedirs(OUTPUT_FOLDER)
        
#     # ---------------------------------------------------------
#     # 1. Scan Plot GDB
#     # ---------------------------------------------------------
#     print(f"Scanning Plot GDB: {INPUT_PLOT_GDB}...")
#     plot_lookup = {}
    
#     try:
#         layers = fiona.listlayers(INPUT_PLOT_GDB)
#         for layer_name in layers:
#             if len(layer_name) >= 4:
#                 suffix = layer_name[-4:]
#                 plot_lookup[suffix] = layer_name
#         print(f"DEBUG: Found {len(plot_lookup)} valid plot layers.")
#     except Exception as e:
#         print(f"Error reading Plot GDB: {e}")
#         return

#     # ---------------------------------------------------------
#     # 2. Load VARI Layer
#     # ---------------------------------------------------------
#     print("Loading VARI layer into memory...")
#     try:
#         gdf_vari_master = gpd.read_file(VARI_GDB_PATH, layer=VARI_LAYER_NAME, engine='pyogrio')
#     except Exception as e:
#         print(f"Error loading VARI layer: {e}")
#         return

#     # ---------------------------------------------------------
#     # 3. Process YOLO Files
#     # ---------------------------------------------------------
#     yolo_files = glob.glob(os.path.join(INPUT_YOLO_FOLDER, "*.shp"))
#     print(f"Found {len(yolo_files)} YOLO files. Starting Loop...\n")

#     for fp in yolo_files:
#         filename = os.path.basename(fp)
#         yolo_name = os.path.splitext(filename)[0]
        
#         try:
#             if len(yolo_name) < 4:
#                 print(f"Skipping {yolo_name}: Filename too short.")
#                 continue
            
#             suffix = yolo_name[-4:]
            
#             if suffix not in plot_lookup:
#                 continue
            
#             plot_layer_name = plot_lookup[suffix]
#             print(f"Processing: {yolo_name} matches Plot {plot_layer_name}")
            
#             # Load Data
#             gdf_yolo = gpd.read_file(fp)
#             # Read Plot with Attributes
#             gdf_plot = gpd.read_file(INPUT_PLOT_GDB, layer=plot_layer_name, engine='pyogrio')
            
#             if gdf_plot.empty:
#                 print(f"  [!] Plot layer empty. Copying original.")
#                 gdf_yolo.to_file(os.path.join(OUTPUT_FOLDER, filename))
#                 continue

#             # Align CRS
#             target_crs = gdf_plot.crs
#             if gdf_yolo.crs != target_crs:
#                 gdf_yolo = gdf_yolo.to_crs(target_crs)
            
#             # Clip VARI
#             if gdf_vari_master.crs != target_crs:
#                 gdf_plot_vari_crs = gdf_plot.to_crs(gdf_vari_master.crs)
#                 gdf_vari_clipped = gpd.clip(gdf_vari_master, gdf_plot_vari_crs)
#                 if not gdf_vari_clipped.empty:
#                     gdf_vari_clipped = gdf_vari_clipped.to_crs(target_crs)
#             else:
#                 gdf_vari_clipped = gpd.clip(gdf_vari_master, gdf_plot)

#             if gdf_vari_clipped.empty:
#                 print("  No VARI data in this plot. Saving original.")
#                 gdf_yolo.to_file(os.path.join(OUTPUT_FOLDER, filename))
#                 continue

#             # Erase
#             gdf_erased = gpd.overlay(gdf_vari_clipped, gdf_yolo, how='difference')

#             if gdf_erased.empty:
#                 print("  No missing plants found.")
#                 # We still want to join plot attributes even if no VARI was added!
#                 gdf_final_combined = gdf_yolo
#             else:
#                 # Subdivide
#                 gdf_erased['area_sqm'] = gdf_erased.geometry.area
#                 small_polys = gdf_erased[gdf_erased['area_sqm'] <= SUBDIVIDE_THRESHOLD].copy()
#                 large_polys = gdf_erased[gdf_erased['area_sqm'] > SUBDIVIDE_THRESHOLD].copy()
                
#                 new_chunks = []
#                 if not large_polys.empty:
#                     print(f"  Subdividing {len(large_polys)} large polygons...")
#                     for geom in large_polys.geometry:
#                         chunks = subdivide_polygon(geom, SUBDIVIDE_THRESHOLD)
#                         new_chunks.extend(chunks)
                
#                 if new_chunks:
#                     gdf_subdivided = gpd.GeoDataFrame(geometry=new_chunks, crs=target_crs)
#                     gdf_final_vari = pd.concat([small_polys, gdf_subdivided], ignore_index=True)
#                 else:
#                     gdf_final_vari = small_polys

#                 # Combine
#                 gdf_final_combined = pd.concat([gdf_yolo, gdf_final_vari], ignore_index=True)

#             # --- ITERATIVE ELIMINATION ---
#             current_gdf = gdf_final_combined
#             for i in range(ELIMINATION_PASSES):
#                 exploded_check = current_gdf.explode(index_parts=False)
#                 num_small = len(exploded_check[exploded_check.geometry.area < MIN_AREA_THRESHOLD])
#                 if num_small == 0:
#                     break
#                 current_gdf = eliminate_pass(current_gdf, MIN_AREA_THRESHOLD, i+1)

#             # --- NEW: JOIN PLOT ATTRIBUTES ---
#             print("    - Joining Plot Attributes to Plants...")
#             # Perform Spatial Join: Plants (left) + Plot (right)
#             # This attaches columns from gdf_plot to current_gdf
#             gdf_attributed = gpd.sjoin(current_gdf, gdf_plot, how='left', predicate='intersects')
            
#             # Clean up the index column created by sjoin
#             if 'index_right' in gdf_attributed.columns:
#                 gdf_attributed = gdf_attributed.drop(columns=['index_right'])

#             # Save
#             out_path = os.path.join(OUTPUT_FOLDER, filename)
#             gdf_attributed.to_file(out_path)
#             print(f"  Success: Saved to {out_path}")

#         except Exception as e:
#             print(f"FAILED on {yolo_name}: {e}")
#             continue

#     print("\n=== Processing Complete ===")

# if __name__ == "__main__":
#     main()

In [5]:
import os
import math
import glob
import geopandas as gpd
import pandas as pd
from shapely.geometry import box
import fiona
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# ================= CONFIGURATION =================
# 1. Folder containing your YOLO shapefiles
#INPUT_YOLO_FOLDER = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\outputRESULTS"
INPUT_YOLO_FOLDER = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\outputRESULTS"

# 2. Path to the VARI layer
#VARI_GDB_PATH = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot24\vari2024.gdb"
#VARI_GDB_PATH = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\plot2022\VARI2022.gdb"
#VARI_GDB_PATH = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\plot21mrc\vari21.gdb"
#VARI_GDB_PATH = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\plot2022\vari_mrc22.gdb"
#VARI_GDB_PATH = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\Vari2025.gdb"
#VARI_GDB_PATH = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025Canada\Varicanada25.gdb"
VARI_GDB_PATH = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2024Canada\Vari24canada.gdb"

#VARI_LAYER_NAME = "VariIndex2024v3"
#VARI_LAYER_NAME = "varisevrec25"
#VARI_LAYER_NAME = "Vari25canada"
VARI_LAYER_NAME = "VariCanada24"

# 3. Path to the Plot Boundaries
#INPUT_PLOT_GDB = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot24\CorrectedPlotDimensions.gdb"
#INPUT_PLOT_GDB = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\plot2022\CorrectedPlotDimensions.gdb"
#INPUT_PLOT_GDB = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\plot21mrc\New File Geodatabase.gdb"
#INPUT_PLOT_GDB = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\plot2022\MRC2022.gdb"
#INPUT_PLOT_GDB = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025\25plots_geometry.gdb"
#INPUT_PLOT_GDB = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2025Canada\New File Geodatabase.gdb"
INPUT_PLOT_GDB = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2024Canada\New File Geodatabase.gdb"



# 4. Output Folder
#OUTPUT_FOLDER = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\outputRESULTS\correctedDetections\2022MRC"
#OUTPUT_FOLDER = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\outputRESULTS\correctedDetections\2025\again"
#OUTPUT_FOLDER = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\outputRESULTS\correctedDetections\Canada25"
OUTPUT_FOLDER = r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\outputRESULTS\correctedDetections\Canada24"


# 5. Area settings
SUBDIVIDE_THRESHOLD = 0.028 # Subdivide anything larger than this
MIN_AREA_THRESHOLD = 0.0016  # Eliminate anything smaller than this
ELIMINATION_PASSES = 3       # How many times to try eliminating small polygons      
# =================================================

def subdivide_polygon(geometry, threshold_area):
    """
    Splits a large polygon into smaller chunks using a grid (fishnet) approach.
    """
    if geometry.area <= threshold_area:
        return [geometry]
    
    cell_size = math.sqrt(threshold_area)
    minx, miny, maxx, maxy = geometry.bounds
    chunks = []
    
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            grid_cell = box(x, y, x + cell_size, y + cell_size)
            chunk = geometry.intersection(grid_cell)
            
            if not chunk.is_empty:
                if chunk.geom_type == 'MultiPolygon':
                    chunks.extend(list(chunk.geoms))
                elif chunk.geom_type in ['Polygon', 'GeometryCollection']:
                    chunks.append(chunk)
            y += cell_size
        x += cell_size
        
    return chunks

def eliminate_pass(gdf, min_area, pass_num):
    """
    Single pass of elimination: 
    1. Explodes to single parts.
    2. Merges small polygons into nearest large neighbor.
    3. Preserves attributes of the 'Parent' (Large) polygon.
    """
    # Explode to ensure we analyze individual pieces
    gdf_exploded = gdf.explode(index_parts=False).reset_index(drop=True)
    
    gdf_exploded['temp_area'] = gdf_exploded.geometry.area
    
    # Identify Small (to be eaten) vs Large (eaters)
    small_polys = gdf_exploded[gdf_exploded['temp_area'] < min_area].copy()
    large_polys = gdf_exploded[gdf_exploded['temp_area'] >= min_area].copy()
    
    if small_polys.empty:
        return gdf_exploded.drop(columns=['temp_area'])
    
    if large_polys.empty:
        return gdf_exploded.drop(columns=['temp_area'])

    print(f"    - Pass {pass_num}: Eliminating {len(small_polys)} tiny polygons...")

    # Assign a unique merge_id to the Large polygons (Parents)
    large_polys['merge_id'] = range(len(large_polys))
    
    # Find nearest Large neighbor for every Small polygon
    # We only care about the geometry and merge_id for the join
    joined = gpd.sjoin_nearest(small_polys, large_polys[['geometry', 'merge_id']], how='left', distance_col='dist')
    
    # The small polygons now know which 'merge_id' they belong to
    small_polys_mapped = joined[['geometry', 'merge_id']].dropna()
    
    # Combine the lists (Large + Mapped Small)
    combined = pd.concat([large_polys[['geometry', 'merge_id']], small_polys_mapped])
    
    # Perform the geometric dissolve
    dissolved_geom = combined.dissolve(by='merge_id')
    
    # --- ATTRIBUTE RESTORATION ---
    # The dissolve destroyed attributes. We must join them back from the original Large parents.
    # merge_id in 'large_polys' corresponds to the index of 'dissolved_geom'
    
    # Get original attributes from large_polys (dropping temp columns)
    cols_to_keep = [c for c in large_polys.columns if c not in ['geometry', 'temp_area', 'merge_id']]
    attributes = large_polys.set_index('merge_id')[cols_to_keep]
    
    # Join attributes back to the new dissolved geometries
    final_gdf = dissolved_geom.join(attributes)
    
    return final_gdf.reset_index(drop=True)

def main():
    print("=== Starting Plant Refinement (YOLO Attrs + Fill 0) ===")
    
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)
        
    # ---------------------------------------------------------
    # 1. Scan Plot GDB
    # ---------------------------------------------------------
    print(f"Scanning Plot GDB: {INPUT_PLOT_GDB}...")
    plot_lookup = {}
    
    try:
        layers = fiona.listlayers(INPUT_PLOT_GDB)
        for layer_name in layers:
            if len(layer_name) >= 4:
                suffix = layer_name[-4:]
                plot_lookup[suffix] = layer_name
        print(f"DEBUG: Found {len(plot_lookup)} valid plot layers.")
    except Exception as e:
        print(f"Error reading Plot GDB: {e}")
        return

    # ---------------------------------------------------------
    # 2. Load VARI Layer
    # ---------------------------------------------------------
    print("Loading VARI layer into memory...")
    try:
        gdf_vari_master = gpd.read_file(VARI_GDB_PATH, layer=VARI_LAYER_NAME, engine='pyogrio')
    except Exception as e:
        print(f"Error loading VARI layer: {e}")
        return

    # ---------------------------------------------------------
    # 3. Process YOLO Files
    # ---------------------------------------------------------
    yolo_files = glob.glob(os.path.join(INPUT_YOLO_FOLDER, "*.shp"))
    print(f"Found {len(yolo_files)} YOLO files. Starting Loop...\n")

    for fp in yolo_files:
        filename = os.path.basename(fp)
        yolo_name = os.path.splitext(filename)[0]
        
        try:
            if len(yolo_name) < 4:
                print(f"Skipping {yolo_name}: Filename too short.")
                continue
            
            suffix = yolo_name[-4:]
            
            if suffix not in plot_lookup:
                continue
            
            plot_layer_name = plot_lookup[suffix]
            print(f"Processing: {yolo_name} matches Plot {plot_layer_name}")
            
            # Load Data
            gdf_yolo = gpd.read_file(fp)
            gdf_plot = gpd.read_file(INPUT_PLOT_GDB, layer=plot_layer_name, engine='pyogrio')
            
            if gdf_plot.empty:
                print(f"  [!] Plot layer empty. Copying original.")
                gdf_yolo.to_file(os.path.join(OUTPUT_FOLDER, filename))
                continue

            # Align CRS
            target_crs = gdf_plot.crs
            if gdf_yolo.crs != target_crs:
                gdf_yolo = gdf_yolo.to_crs(target_crs)
            
            # Clip VARI
            if gdf_vari_master.crs != target_crs:
                gdf_plot_vari_crs = gdf_plot.to_crs(gdf_vari_master.crs)
                gdf_vari_clipped = gpd.clip(gdf_vari_master, gdf_plot_vari_crs)
                if not gdf_vari_clipped.empty:
                    gdf_vari_clipped = gdf_vari_clipped.to_crs(target_crs)
            else:
                gdf_vari_clipped = gpd.clip(gdf_vari_master, gdf_plot)

            if gdf_vari_clipped.empty:
                print("  No VARI data in this plot. Saving original.")
                gdf_yolo.to_file(os.path.join(OUTPUT_FOLDER, filename))
                continue

            # Erase
            gdf_erased = gpd.overlay(gdf_vari_clipped, gdf_yolo, how='difference')

            if gdf_erased.empty:
                print("  No missing plants found.")
                gdf_final_combined = gdf_yolo
            else:
                # Subdivide
                gdf_erased['area_sqm'] = gdf_erased.geometry.area
                small_polys = gdf_erased[gdf_erased['area_sqm'] <= SUBDIVIDE_THRESHOLD].copy()
                large_polys = gdf_erased[gdf_erased['area_sqm'] > SUBDIVIDE_THRESHOLD].copy()
                
                new_chunks = []
                if not large_polys.empty:
                    print(f"  Subdividing {len(large_polys)} large polygons...")
                    for geom in large_polys.geometry:
                        chunks = subdivide_polygon(geom, SUBDIVIDE_THRESHOLD)
                        new_chunks.extend(chunks)
                
                if new_chunks:
                    gdf_subdivided = gpd.GeoDataFrame(geometry=new_chunks, crs=target_crs)
                    gdf_final_vari = pd.concat([small_polys, gdf_subdivided], ignore_index=True)
                else:
                    gdf_final_vari = small_polys

                # Combine VARI + YOLO (YOLO attributes exist here, VARI has NaNs)
                gdf_final_combined = pd.concat([gdf_yolo, gdf_final_vari], ignore_index=True)

            # --- ITERATIVE ELIMINATION (With Attribute Preservation) ---
            current_gdf = gdf_final_combined
            for i in range(ELIMINATION_PASSES):
                exploded_check = current_gdf.explode(index_parts=False)
                num_small = len(exploded_check[exploded_check.geometry.area < MIN_AREA_THRESHOLD])
                if num_small == 0:
                    break
                current_gdf = eliminate_pass(current_gdf, MIN_AREA_THRESHOLD, i+1)

            # --- NEW: HANDLE CLASS VALUES ---
            # Check if 'cls' column exists (standard in YOLO export)
            # If not, try to find a column that looks like a class ID, or skip
            target_col = 'cls'
            
            if target_col not in current_gdf.columns:
                # Fallback: Check for 'class', 'Class', 'gridcode' or create 'cls'
                possible_cols = [c for c in current_gdf.columns if c.lower() in ['class', 'grid_code', 'label']]
                if possible_cols:
                    target_col = possible_cols[0]
                else:
                    print(f"    - Warning: 'cls' column not found. Creating it.")
                    current_gdf[target_col] = None # Create empty

            # Fill NaN/None with 0
            print(f"    - Filling missing '{target_col}' values with 0...")
            current_gdf[target_col] = current_gdf[target_col].fillna(0)
            
            # Also ensure it's integer type if possible
            try:
                current_gdf[target_col] = current_gdf[target_col].astype(int)
            except:
                pass # Keep as is if conversion fails

            # --- JOIN PLOT ATTRIBUTES ---
            print("    - Joining Plot Attributes...")
            gdf_attributed = gpd.sjoin(current_gdf, gdf_plot, how='left', predicate='intersects')
            
            if 'index_right' in gdf_attributed.columns:
                gdf_attributed = gdf_attributed.drop(columns=['index_right'])

            # Save
            out_path = os.path.join(OUTPUT_FOLDER, filename)
            gdf_attributed.to_file(out_path)
            print(f"  Success: Saved to {out_path}")

        except Exception as e:
            print(f"FAILED on {yolo_name}: {e}")
            continue

    print("\n=== Processing Complete ===")

if __name__ == "__main__":
    main()

=== Starting Plant Refinement (YOLO Attrs + Fill 0) ===
Scanning Plot GDB: C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\Plot2024Canada\New File Geodatabase.gdb...
DEBUG: Found 1 valid plot layers.
Loading VARI layer into memory...
Found 26 YOLO files. Starting Loop...

Processing: Canada24116 matches Plot P24116
    - Pass 1: Eliminating 694802 tiny polygons...
    - Pass 2: Eliminating 686055 tiny polygons...
    - Pass 3: Eliminating 686643 tiny polygons...
    - Filling missing 'cls' values with 0...
    - Joining Plot Attributes...
  Success: Saved to C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\outputRESULTS\correctedDetections\Canada24\Canada24116.shp

=== Processing Complete ===
